## Qual a previsão da vazão de saída para as próximas 24h para um determinado dia e horário?

In [1]:
from xgboost import XGBRegressor
import numpy as np 
from sklearn.metrics import mean_squared_error, mean_absolute_error
from xgboost import callback
from sklearn.model_selection import GridSearchCV, KFold
import plotly.express as px
from pathlib import Path
import pandas as pd 
import pickle

In [4]:
df_path = Path("../data/curated_data/water_consumption_curated_1.parquet")
df = pd.read_parquet(df_path)
df.head()

,timestamp,flow_in_(l/s),reservoir_level_(%),pressure_(mca),gmb_1_is_on,gmb_2_is_on,reservoir_level_liters,time_passed_seconds,liters_entered,flow_out_(l/s)
0,2023-03-17 11:27:06,68.59,29.86,38.20,0,1,298600.0,NaN,NaN,NaN
1,2023-03-17 12:28:56,66.05,35.86,38.20,0,1,358600.0,3710.0,60000.0,49.877493
2,2023-03-17 12:31:26,65.64,36.16,38.06,0,1,361600.0,150.0,3000.0,45.640000
3,2023-03-17 12:33:56,65.64,36.50,38.03,0,1,365000.0,150.0,3400.0,42.973333
4,2023-03-17 12:36:26,65.64,36.80,38.17,0,1,368000.0,150.0,3000.0,45.640000


In [5]:
df["year"] = df["timestamp"].dt.year
df["month"] = df["timestamp"].dt.month
df['week_of_year'] = df['timestamp'].dt.isocalendar().week
df['day_of_week'] = df['timestamp'].dt.dayofweek
df["day"] = df["timestamp"].dt.day
df["hour"] = df["timestamp"].dt.hour
df["second"] = df["timestamp"].dt.second

df = df.dropna(subset=["flow_out_(l/s)"])
df.head()


,timestamp,flow_in_(l/s),reservoir_level_(%),pressure_(mca),gmb_1_is_on,gmb_2_is_on,reservoir_level_liters,time_passed_seconds,liters_entered,flow_out_(l/s),year,month,week_of_year,day_of_week,day,hour,second
1,2023-03-17 12:28:56,66.05,35.86,38.20,0,1,358600.0,3710.0,60000.0,49.877493,2023,3,11,4,17,12,56
2,2023-03-17 12:31:26,65.64,36.16,38.06,0,1,361600.0,150.0,3000.0,45.640000,2023,3,11,4,17,12,26
3,2023-03-17 12:33:56,65.64,36.50,38.03,0,1,365000.0,150.0,3400.0,42.973333,2023,3,11,4,17,12,56
4,2023-03-17 12:36:26,65.64,36.80,38.17,0,1,368000.0,150.0,3000.0,45.640000,2023,3,11,4,17,12,26
5,2023-03-17 12:38:56,65.24,36.80,38.17,0,1,368000.0,150.0,0.0,65.240000,2023,3,11,4,17,12,56


In [8]:
df.to_parquet("../data/curated_data/water_consumption_curated_2.parquet")

In [9]:
X = df[["flow_in_(l/s)", "reservoir_level_(%)", "pressure_(mca)", "gmb_1_is_on", "gmb_2_is_on", "reservoir_level_liters", 
        "time_passed_seconds", "liters_entered", "year", "month", "week_of_year", "day_of_week", "day", "hour", "second"]]

y = df["flow_out_(l/s)"]

In [10]:
# train test split
train_size = int(0.8 * len(X))
x_train, x_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [11]:
class LearningRateDecay(callback.TrainingCallback):
    def __init__(self, initial_lr=0.01, decay_rate=0.1, decay_steps=1000):
        self.initial_lr = initial_lr
        self.decay_rate = decay_rate
        self.decay_steps = decay_steps

    def after_iteration(self, model, epoch, evals_log):
        new_lr = self.initial_lr * self.decay_rate ** (epoch / self.decay_steps)
        model.set_param('learning_rate', new_lr)
        return False

# Instantiate the XGBRegressor with the custom callback
model = XGBRegressor(
    n_estimators=100000, 
    learning_rate=0.01, 
    early_stopping_rounds=100,
    callbacks=[LearningRateDecay(initial_lr=0.01, decay_rate=0.1, decay_steps=1000)]
)

# Set up cross-validation
cv = KFold(n_splits=5)

# Define parameter grid for GridSearchCV
params = {
    'n_estimators': [100, 300, 500],
    'max_depth': [3, 5, 10, 14]
}

# Instantiate GridSearchCV
clf = GridSearchCV(estimator=model, param_grid=params, cv=cv)

# Train the model (no need to pass callbacks in fit)
clf.fit(
    x_train, y_train, 
    eval_set=[(x_train, y_train), (x_test, y_test)], 
    verbose=100
)

[0]	validation_0-rmse:101.21552	validation_1-rmse:120.47506
[99]	validation_0-rmse:92.75784	validation_1-rmse:108.04932
[0]	validation_0-rmse:100.80451	validation_1-rmse:120.05225
[99]	validation_0-rmse:76.72476	validation_1-rmse:92.69378
[0]	validation_0-rmse:100.77947	validation_1-rmse:120.00444
[99]	validation_0-rmse:75.60098	validation_1-rmse:90.73234
[0]	validation_0-rmse:100.77277	validation_1-rmse:120.03618
[99]	validation_0-rmse:75.97132	validation_1-rmse:90.67068
[0]	validation_0-rmse:100.76494	validation_1-rmse:120.08916
[99]	validation_0-rmse:74.43951	validation_1-rmse:91.24001
[0]	validation_0-rmse:101.21552	validation_1-rmse:120.47506
[100]	validation_0-rmse:92.67874	validation_1-rmse:107.92678
[200]	validation_0-rmse:88.73734	validation_1-rmse:102.07513
[299]	validation_0-rmse:86.73151	validation_1-rmse:99.44451
[0]	validation_0-rmse:100.80451	validation_1-rmse:120.05225
[100]	validation_0-rmse:76.64373	validation_1-rmse:92.60940
[200]	validation_0-rmse:72.15272	validatio

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=[<__main__.LearningRateDecay object at 0x175b77350>],
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=100,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=...
                                    learning_rate=0.01, max_bin=None,
                                    max_cat_threshold=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=100000,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'max_depth': [3, 5, 10, 14],
                         'n_estimators': [100, 300, 500]})

The numbers within square brackets, such as `[ ]`, in the log output represent the iteration number or boosting round during the training process of the `XGBRegressor` model. XGBoost utilizes a gradient boosting framework, which builds the model sequentially by adding one tree at a time and optimizing the model at each iteration. Each line in the log shows the root mean square error (RMSE) for both the training set (`validation_0-rmse`) and the validation set (`validation_1-rmse`) at the given iteration. For instance, `[0] validation_0-rmse:101.21552 validation_1-rmse:120.47506` indicates that at the first iteration (iteration 0), the RMSE for the training set is 101.21552, and the RMSE for the validation set is 120.47506. As training progresses, these RMSE values typically decrease, reflecting the model's improvement. However, they may plateau or even increase if the model begins to overfit the training data. Therefore, monitoring these metrics across iterations helps in understanding the model's performance and potential overfitting.

In [12]:
def plot_diagnostics(y_test, y_pred):
    print(f'MSE: {round(mean_squared_error(y_true=y_test, y_pred=y_pred),2)}')
    print(f'RMSE: {round(np.sqrt(mean_squared_error(y_true=y_test, y_pred=y_pred)),2)}')
    print(f'MAE: {round(mean_absolute_error(y_true=y_test, y_pred=y_pred),2)}')
    
plot_diagnostics(y_test, clf.predict(x_test))

MSE: 4342.66
RMSE: 65.9
MAE: 3.37


In [17]:
df["flow_out_forecast"] = clf.predict(X)
df.head()

,timestamp,flow_in_(l/s),reservoir_level_(%),pressure_(mca),gmb_1_is_on,gmb_2_is_on,reservoir_level_liters,time_passed_seconds,liters_entered,flow_out_(l/s),year,month,week_of_year,day_of_week,day,hour,second,flow_out_forecast
1,2023-03-17 12:28:56,66.05,35.86,38.20,0,1,358600.0,3710.0,60000.0,49.877493,2023,3,11,4,17,12,56,48.892666
2,2023-03-17 12:31:26,65.64,36.16,38.06,0,1,361600.0,150.0,3000.0,45.640000,2023,3,11,4,17,12,26,44.910580
3,2023-03-17 12:33:56,65.64,36.50,38.03,0,1,365000.0,150.0,3400.0,42.973333,2023,3,11,4,17,12,56,42.245819
4,2023-03-17 12:36:26,65.64,36.80,38.17,0,1,368000.0,150.0,3000.0,45.640000,2023,3,11,4,17,12,26,44.910580
5,2023-03-17 12:38:56,65.24,36.80,38.17,0,1,368000.0,150.0,0.0,65.240000,2023,3,11,4,17,12,56,63.653023


In [18]:
df.to_parquet("../data/curated_data/water_consumption_curated_2.parquet")

In [13]:
with open('../models/xgb_flow_out_forecast.pkl', 'wb') as f:
    pickle.dump(clf, f)

In [14]:
print(df.head())

            timestamp  flow_in_(l/s)  reservoir_level_(%)  pressure_(mca)  \
1 2023-03-17 12:28:56          66.05                35.86           38.20   
2 2023-03-17 12:31:26          65.64                36.16           38.06   
3 2023-03-17 12:33:56          65.64                36.50           38.03   
4 2023-03-17 12:36:26          65.64                36.80           38.17   
5 2023-03-17 12:38:56          65.24                36.80           38.17   

   gmb_1_is_on  gmb_2_is_on  reservoir_level_liters  time_passed_seconds  \
1            0            1                358600.0               3710.0   
2            0            1                361600.0                150.0   
3            0            1                365000.0                150.0   
4            0            1                368000.0                150.0   
5            0            1                368000.0                150.0   

   liters_entered  flow_out_(l/s)  year  month  week_of_year  day_of_week  \
1  

In [15]:
results_df = pd.DataFrame({
    'timestamp': df['timestamp'].iloc[train_size:].reset_index(drop=True),
    'Actual': y_test.reset_index(drop=True),
    'Predicted': pd.Series(clf.predict(x_test))
})

# Plotar os resultados
fig = px.line(results_df, x='timestamp', y=['Actual', 'Predicted'],
              labels={'value': 'Flow Out (l/s)', 'timestamp': 'Timestamp'},
              title='Comparação entre valores reais e previstos')

fig.show()

In [16]:
print(results_df.head(50))

             timestamp     Actual  Predicted
0  2024-01-15 03:47:42  26.666667  27.000076
1  2024-01-15 03:52:42   8.000000   9.314432
2  2024-01-15 03:55:12  26.666667  27.000076
3  2024-01-15 04:00:12  12.000000  13.204606
4  2024-01-15 04:02:42  20.000000  20.623121
5  2024-01-15 04:05:12  16.000000  17.421682
6  2024-01-15 04:07:42   0.000000   1.711583
7  2024-01-15 04:10:12  26.666667  27.000076
8  2024-01-15 04:12:42  17.333333  17.421682
9  2024-01-15 04:15:12  20.000000  20.623121
10 2024-01-15 04:17:42  16.000000  17.421682
11 2024-01-15 04:20:12   0.000000   1.711583
12 2024-01-15 04:22:42  26.666667  27.000076
13 2024-01-15 04:25:12  24.000000  24.528915
14 2024-01-15 04:27:42   0.000000   1.711583
15 2024-01-15 04:32:42  23.333333  23.919022
16 2024-01-15 04:35:12  26.666667  27.000076
17 2024-01-15 04:37:42   0.000000   1.711583
18 2024-01-15 04:40:12  17.333333  17.421682
19 2024-01-15 04:42:42  22.666667  23.281191
20 2024-01-15 04:45:13   0.000000   1.711583
21 2024-01